# Optical flow random image denoising

In [ ]:
local_debug = True

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
#from ipywidgets import *
import cv2
import time
#import kernels
from skimage import io as skimage_io
import mrcfile
from collections import namedtuple

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
import denoising.image.OF_random as denoising

In [ ]:
Args = namedtuple("args", "input")
args = Args("FoilHole.mrc")

In [ ]:
%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="11lmIglqAgIgTRS3YZZccDM-ma2bI-CXr"
    wget --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O $OUTPUT_FILENAME 2> /dev/null
fi
set +x

In [ ]:
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
padded_noisy = np.zeros(shape=(noisy.shape[0]+32, noisy.shape[1]+32), dtype=np.uint8)
padded_noisy[16:-16, 16:-16] = noisy
noisy = padded_noisy

In [ ]:
plt.imshow(noisy, cmap="gray")

In [ ]:
denoiser = denoising.Monochrome_Denoiser(
    logger,
    pyramid_levels=3,
    window_side = 5,
    N_poly = 5,
    num_iterations = 3
)

In [ ]:
denoised, _ = denoiser.filter(noisy, None, N_iters=20, RS_sigma=0.75)
#denoised, _ = denoiser.filter(noisy, GT, N_iters=80, RS_sigma=5.5)

In [ ]:
np.min(denoised)

In [ ]:
np.max(denoised)

In [ ]:
denoised = np.clip(denoised, a_min=0, a_max=255)

In [ ]:
denoised = np.round(denoised).astype(np.uint8)

In [ ]:
np.min(denoised)

In [ ]:
np.max(denoised)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy, cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy, denoised)})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[200:400,800:1000], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[200:400,800:1000], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[500:700,1200:], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[500:700,1200:], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
np.mean(noisy)

In [ ]:
np.mean(denoised)